<a href="https://colab.research.google.com/github/A00785001/TC5035/blob/main/01_Camera_Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROS Bag Camera Data Reader
This notebook reads and displays camera data from ROS bag files

In [ ]:
# Import libraries
import rosbag
from cv_bridge import CvBridge
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Initialize CV Bridge
bridge = CvBridge()
print("Libraries loaded successfully!")

In [ ]:
# Path to your ROS bag file
bag_file = "your_bag_file.bag"

In [ ]:
# Open the bag
bag = rosbag.Bag(bag_file)
print(f"Opened bag file: {bag_file}")

In [ ]:
# List all topics in the bag
print("Topics in bag:")
for topic, msg_type, _ in bag.get_type_and_topic_info().topics.items():
    print(f"  {topic}: {msg_type}")

In [ ]:
# Specify your camera topic
camera_topic = "/camera/image_raw"
print(f"Using camera topic: {camera_topic}")

In [ ]:
# Read images from the bag
images = []
timestamps = []

print(f"Reading images from topic: {camera_topic}")
for topic, msg, t in bag.read_messages(topics=[camera_topic]):
    # Convert ROS Image message to OpenCV image
    cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

    # Convert BGR to RGB for matplotlib
    rgb_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)

    images.append(rgb_image)
    timestamps.append(t.to_sec())

    # Limit to first 10 images for quick preview
    if len(images) >= 10:
        break

print(f"Total images read: {len(images)}")
if images:
    print(f"Image shape: {images[0].shape}")
    print(f"Time range: {timestamps[0]:.3f} to {timestamps[-1]:.3f} seconds")

In [ ]:
# Display first image
if images:
    plt.figure(figsize=(10, 8))
    plt.imshow(images[0])
    plt.title(f"First Image - Timestamp: {timestamps[0]}")
    plt.axis('off')
    plt.show()

In [ ]:
# Display multiple images in a grid
if len(images) > 1:
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()

    for i, (img, ts) in enumerate(zip(images[:10], timestamps[:10])):
        axes[i].imshow(img)
        axes[i].set_title(f"t={ts:.2f}s", fontsize=8)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Save an image to file
if images:
    cv2.imwrite('extracted_frame.jpg', cv2.cvtColor(images[0], cv2.COLOR_RGB2BGR))
    print("Saved first frame as 'extracted_frame.jpg'")

In [ ]:
# Close the bag
bag.close()
print("Bag file closed")